In [ ]:
# from socket import AF_INET, socket, SOCK_STREAM
# from threading import Thread
# import os
# import uuid  # To generate unique filenames for voice messages

# HOST = "localhost"
# PORT = 33000
# BUFSIZ = 1024
# ADDR = (HOST, PORT)

# server_socket = socket(AF_INET, SOCK_STREAM)
# server_socket.bind(ADDR)

# def handle_client(client_socket):
#     """Handles a single client connection."""
#     client_socket.send(bytes("Bot: Hey there! Tell me about your day", "utf8"))
#     while True:
#         try:
#             # Receive the message type indicator
#             msg_type = client_socket.recv(BUFSIZ).decode("utf8")
#             if not msg_type:
#                 break

#             if msg_type == "voice":
#                 # Receive the file size
#                 file_size = int(client_socket.recv(BUFSIZ).decode("utf8"))
#                 received_size = 0
#                 file_name = str(uuid.uuid4()) + ".wav"
#                 with open(file_name, "wb") as audio_file:
#                     while received_size < file_size:
#                         data = client_socket.recv(BUFSIZ)
#                         if not data:
#                             break
#                         audio_file.write(data)
#                         received_size += len(data)

#                 print(f"Voice message received and saved as {file_name}.")
#             else:
#                 # Handle text messages
#                 data = client_socket.recv(BUFSIZ).decode("utf8")
#                 if not data:
#                     break
#                 response = "Bot: Received your message!"
#                 print(f"Client: {data}")
#                 client_socket.send(bytes(response, "utf8"))

#         except ConnectionResetError:
#             break

#     client_socket.close()
#     print("Client disconnected.")

# def accept_connections():
#     """Accepts incoming connections."""
#     while True:
#         client_socket, client_address = server_socket.accept()
#         print(f"{client_address} has connected.")
#         Thread(target=handle_client, args=(client_socket,)).start()

# if __name__ == "__main__":
#     server_socket.listen(5)
#     print("Waiting for connection...")
#     accept_connections()


Waiting for connection...
('127.0.0.1', 53654) has connected.
Client: hey
Voice message received and saved as 7e2c3260-0072-436e-b1bf-a6c32f2c8e4f.wav.
Client: {quit}
Client disconnected.
('127.0.0.1', 53672) has connected.
Client disconnected.
('127.0.0.1', 53749) has connected.
Client: hello
Voice message received and saved as 6dc1081f-a26f-4c82-98fe-7b0718f86af3.wav.
Client: {quit}
Client disconnected.


In [2]:
import os
import uuid
import librosa
import numpy as np
from socket import AF_INET, socket, SOCK_STREAM
from threading import Thread
from keras.models import load_model
from keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
import tensorflow as tf


In [ ]:
import os
import uuid
import librosa
import numpy as np
from socket import AF_INET, socket, SOCK_STREAM
from threading import Thread
from keras.models import load_model
from keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler

# Server configuration
HOST = "localhost"
PORT = 33000
BUFSIZ = 1024
ADDR = (HOST, PORT)

server_socket = socket(AF_INET, SOCK_STREAM)
server_socket.bind(ADDR)

# Load the trained CNN model
model_path = "C:/Users/REEM/Desktop/Speech/CNN-NoSC.h5"
model = load_model(model_path)
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Load the scaler used during training
scaler = StandardScaler()
scaler.fit(np.random.rand(2376).reshape(-1, 1).T)  # Assuming the scaler was fit on data with shape (2376,)

# Load the responses for each emotion from a file
responses = {}
with open("responses.txt", "r") as f:
    for line in f:
        emotion, response = line.strip().split(": ", 1)
        responses[emotion] = response

def extract_features(file_name):
    """Extract features from audio file for prediction."""
    try:
        X, sample_rate = librosa.load(file_name, sr=None)
        if len(X) < 2048:
            raise ValueError("Input signal length too short for valid MFCC extraction.")
        
        mfccs = librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40)
        padded_mfccs = np.zeros((40, 60))  # Ensuring a consistent shape

        # If the MFCCs are shorter, pad them; if longer, truncate
        if mfccs.shape[1] < 60:
            padded_mfccs[:, :mfccs.shape[1]] = mfccs
        else:
            padded_mfccs = mfccs[:, :60]

        mfccs_processed = np.mean(padded_mfccs.T, axis=0)
        
        # Pad to ensure a consistent length of 2376 if needed
        if len(mfccs_processed) < 2376:
            padded_features = np.pad(mfccs_processed, (0, 2376 - len(mfccs_processed)), 'constant')
        else:
            padded_features = mfccs_processed[:2376]
        
        # Log raw features for debugging
        print(f"Raw MFCCs: {mfccs_processed}")

        return padded_features
    except Exception as e:
        print(f"Error in feature extraction: {e}")
        return None

def handle_client(client_socket):
    """Handles a single client connection."""
    client_socket.send(bytes("Bot: Hey there! Tell me about your day", "utf8"))
    while True:
        try:
            # Receive the message type indicator
            msg_type = client_socket.recv(BUFSIZ).decode("utf8")
            if not msg_type:
                break

            if msg_type == "text":
                # Handle text messages
                data = client_socket.recv(BUFSIZ).decode("utf8")
                if not data:
                    break
                print(f"Client: {data}")
                response = "Bot: Received your message!"
                client_socket.send(bytes(response, "utf8"))

            elif msg_type == "voice":
                # Receive the file size
                file_size = int(client_socket.recv(BUFSIZ).decode("utf8"))
                received_size = 0
                file_name = os.path.join("C:/Users/REEM/Desktop/Speech", str(uuid.uuid4()) + ".wav")
                
                # Create directory if it doesn't exist
                if not os.path.exists(os.path.dirname(file_name)):
                    os.makedirs(os.path.dirname(file_name))
                
                with open(file_name, "wb") as audio_file:
                    while received_size < file_size:
                        data = client_socket.recv(BUFSIZ)
                        if not data:
                            break
                        audio_file.write(data)
                        received_size += len(data)

                print(f"Voice message received and saved as {file_name}.")

                # Ensure the file was saved
                if not os.path.exists(file_name):
                    print(f"Error: File {file_name} was not saved correctly.")
                    client_socket.send(bytes("Bot: There was an error saving your voice message.", "utf8"))
                    continue

                # Extract features and predict emotion
                features = extract_features(file_name)
                if features is not None:
                    features = scaler.transform(features.reshape(1, -1))  # Scale the features
                    print(f"Scaled features shape: {features.shape}")  # Logging the shape of scaled features
                    features = np.expand_dims(features, axis=2)  # Add channel dimension
                    print(f"Features shape after adding channel dimension: {features.shape}")  # Log shape after adding channel dimension

                    if features.shape[1] != 2376:
                        raise ValueError(f"Unexpected feature shape: {features.shape}")

                    prediction = model.predict(features)
                    print(f"Prediction: {prediction}")  # Log the raw prediction

                    emotion = np.argmax(prediction, axis=1)
                    emotion_label = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad'][emotion[0]]
                    print(f"Predicted emotion: {emotion_label}")  # Log the predicted emotion

                    response = f"Bot: I sense that you are feeling {emotion_label}. {responses[emotion_label]}"
                else:
                    response = "Bot: There was an error processing your voice message."

                client_socket.send(bytes(response, "utf8"))

                # Clean up the temporary audio file
                os.remove(file_name)

        except ConnectionResetError:
            break
        except ValueError as ve:
            print(f"ValueError: {ve}")
            client_socket.send(bytes("Bot: There was an error processing your voice message.", "utf8"))

    client_socket.close()
    print("Client disconnected.")

def accept_connections():
    """Accepts incoming connections."""
    while True:
        client_socket, client_address = server_socket.accept()
        print(f"{client_address} has connected.")
        Thread(target=handle_client, args=(client_socket,)).start()

if __name__ == "__main__":
    server_socket.listen(5)
    print("Waiting for connection...")
    accept_connections()


Waiting for connection...
('127.0.0.1', 51522) has connected.
Client disconnected.
('127.0.0.1', 51592) has connected.
Voice message received and saved as C:/Users/REEM/Desktop/Speech\53b46b75-f9dc-4f3b-882a-f3fd190dea70.wav.
Raw MFCCs: [-7.15494690e+02  5.12101479e+01 -1.94267742e-02  1.79550922e+00
  1.30659437e+01  7.09113073e+00  9.21517611e-02 -2.31519893e-01
  5.52787113e+00  8.70570946e+00  6.16100597e+00  4.75880194e+00
  3.75001645e+00  2.13172865e+00  1.68351793e+00  2.36272430e+00
  4.48993206e+00  4.24781227e+00  1.62873232e+00  7.36007690e-01
 -5.43298423e-01 -1.17070735e+00  1.91932261e-01 -2.12979708e-02
  4.69147176e-01  1.59408450e+00  7.48979077e-02  1.20213054e-01
  2.43777919e+00  2.88615346e+00  1.04050374e+00 -1.33023798e+00
 -6.49336576e-01  9.63318765e-01 -1.45674062e+00 -2.48999286e+00
  6.51460230e-01  1.65221524e+00  2.19812952e-02 -1.10289204e+00]
Scaled features shape: (1, 2376)
Features shape after adding channel dimension: (1, 2376, 1)
1/1 ━━━━━━━━━━━━━━━

Exception in thread Thread-7 (handle_client):
Traceback (most recent call last):
  File "D:\anacondaF\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\REEM\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 761, in run_closure
    _threading_Thread_run(self)
  File "D:\anacondaF\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\REEM\AppData\Local\Temp\ipykernel_20576\4118578052.py", line 98, in handle_client
ConnectionAbortedError: [WinError 10053] An established connection was aborted by the software in your host machine


Client: {quit}
('127.0.0.1', 51636) has connected.
Client: Type your messages here.
Voice message received and saved as C:/Users/REEM/Desktop/Speech\d130a3cf-3edb-4770-a863-e184ec38d2c4.wav.
Raw MFCCs: [-5.8357819e+02  1.8359343e+02 -3.5561089e+01  3.6006798e+01
  1.3000227e+01 -9.6997805e+00  1.9952793e+01 -8.5362358e+00
 -7.0136450e-02  9.1637392e+00 -2.3904755e+00  2.4665527e+00
 -1.0340247e+01 -3.2442942e-01  9.1979609e+00 -9.4460154e+00
 -1.4896624e+00  2.1589255e+00 -4.1107469e+00  5.0372229e+00
  1.1200567e+00 -3.8827124e+00 -8.2823239e-02  7.3867097e-02
 -8.1897996e-02 -3.1114261e+00 -3.2207322e+00 -7.9770756e-01
  1.3865715e+00  6.2484579e+00  4.4004230e+00  2.3630025e+00
  4.7392797e+00  7.5975966e-01 -2.9607942e+00 -1.5112518e-01
  3.7931087e+00  4.7226439e+00  2.3327098e+00 -2.7322975e-01]
Scaled features shape: (1, 2376)
Features shape after adding channel dimension: (1, 2376, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 376ms/step
Prediction: [[4.3361859e-32 2.0520161e-01 1.9482688e-35

In [1]:
# !pip install resampy


Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for resampy from https://files.pythonhosted.org/packages/4d/b9/3b00ac340a1aab3389ebcc52c779914a44aadf7b0cb7a3bf053195735607/resampy-0.4.3-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/3.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.1 MB 217.9 kB/s eta 0:00:15
   ---------------------------------------- 0.0/3.1 MB 217.9 kB/s eta 0:00:14
    --------------------------------------- 0.0/3.1 MB 196.9 kB/s eta 0:00:16
   - -------------------------------------- 0.1/3.1 MB 327.3 kB/s eta 0:00:10
   - -------------------------------------- 0.1/3.1 MB 423.5 kB/s eta 0:00:07
   - -------------------------------------- 0.2/3.1 MB 482.7 kB/s eta 0:00:07
   -- ------------------------------------- 0.2/